# Import des librairies

In [1]:
import pandas as pd

# Chargement des données

In [16]:
df = pd.read_csv("C:/Users/remyl/Downloads/test_set.csv")

# Prévisualisation

In [3]:
df

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock
0,0,1,vilnius,21,romanian,0,1,161,46
1,1,1,vilnius,21,romanian,0,1,187,32
2,2,1,vilnius,21,romanian,0,1,279,12
3,3,1,vilnius,21,romanian,0,1,395,10
4,4,1,vilnius,21,romanian,0,1,488,42
...,...,...,...,...,...,...,...,...,...
6639,6639,843,rome,5,irish,0,794,987,1
6640,6640,844,vienna,1,irish,1,794,26,1
6641,6641,844,vienna,1,irish,1,794,263,0
6642,6642,844,vienna,1,irish,1,794,456,0


# Etude des villes

Premièrement nous allons afficher la répartition des villes dans le dataset de test afin de vérifier si elles sont toutes présentes et de manière équitable.

In [4]:
df["city"].value_counts()

amsterdam     1134
paris         1125
madrid        1094
copenhagen     723
rome           721
vilnius        609
vienna         568
sofia          374
valletta       296
Name: city, dtype: int64

Les villes sont effectivement toutes bien présentes mais de manière très déséquilibrée. Ce déséquilibre pourrait être retrouvé dans l'ensemble des données, ou pas. Pour avoir une idée de la réponse je vais utiliser des métadonnées associées aux villes qui pourraient expliquer le nombre d'hotêls que l'API renvoie pour celles-ci.

Dans un premier temps je vais vérifier si la répartition vue ci-dessus correspond à la taille des villes étudiées (en terme de nombre d'habitants).
Les données sont tirées du site : https://www.populationdata.net/palmares/villes/europe/

In [7]:
population_data = {"amsterdam" : 2480394, "paris" : 13114718, "madrid" : 6475872, "copenhagen" : 2931445, "rome" : 4340474, "vilnius" : 673921, "vienna" : 2717459, "sofia" : 1681592, "valletta" : 408750}

Ensuite je suis allé extraire le nombre de recherches Google des termes "hotel" plus nom de la ville. J'ai stocké le nombre de résultats de la requête dans une liste (ordre = same que renvoyé par df["city"]).

In [15]:
nb_url = [258, 705, 328, 69.5, 545, 11.8, 191, 225, 7.47] # unité = 10**6
# chiffres obtenus sur google.com langue = anglais US

J'ai également été sur le site https://www.booking.com/ et récupéré le nombre d'hôtel renvoyé pour une requête dans chaque ville (pas de dates et résa pour 2 adultes).

In [21]:
data_booking = [1061, 4980, 3344, 475, 9029, 651, 2287, 971, 235]

Des données sur les pays peuvent également être retenues comme le PIB par habitant (à voir si c'est également utilisable pour la feature language).

In [25]:
PIB = [48424, 38210, 26239, 58586, 31512, 18072, 45091, 8294, 27738]

In [26]:
l = list(df["city"].value_counts())
df_city = pd.DataFrame(population_data.items(), columns=['city', 'nb_hab'])
df_city["nb_url"] = nb_url
df_city["booking"] = data_booking
df_city["PIB"] = PIB
df_city["nb_val"] = l

In [27]:
df_city

,city,nb_hab,nb_url,booking,PIB,nb_val
0,amsterdam,2480394,258.00,1061,48424,1134
1,paris,13114718,705.00,4980,38210,1125
2,madrid,6475872,328.00,3344,26239,1094
3,copenhagen,2931445,69.50,475,58586,723
4,rome,4340474,545.00,9029,31512,721
5,vilnius,673921,11.80,651,18072,609
6,vienna,2717459,191.00,2287,45091,568
7,sofia,1681592,225.00,971,8294,374
8,valletta,408750,7.47,235,27738,296
